In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('diabetes.csv')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [8]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()

In [11]:
X = scaler.fit_transform(x)

In [12]:
X.shape

(768, 8)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=3)

In [55]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [15]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(X_train,y_train,batch_size=32,epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.6724 - accuracy: 0.6238 - val_loss: 0.6656 - val_accuracy: 0.6688
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6282 - accuracy: 0.6906 - val_loss: 0.6351 - val_accuracy: 0.7013
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5930 - accuracy: 0.7329 - val_loss: 0.6114 - val_accuracy: 0.7013
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.7476 - val_loss: 0.5938 - val_accuracy: 0.7208
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5413 - accuracy: 0.7492 - val_loss: 0.5785 - val_accuracy: 0.7338
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5222 - accuracy: 0.7606 - val_loss: 0.5682 - val_accuracy: 0.7273
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5068 - accuracy: 0.7687 - val_loss: 0.5590 - val_accuracy: 0.7338
Epoch 8/100


In [17]:
# How to select appropricate optimizer
# No. of nodes in a layer
# How to select no. of layers
# All in one model

In [18]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.3 MB/s eta 0:00:00


In [19]:
import kerastuner as kt

<ipython-input-19-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [20]:
#This is for optimizer selection
def build_model(hp):
  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer = optimizer, loss="binary_crossentropy", metrics=['accuracy'])

  return model

In [21]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy',max_trials=5)

In [22]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 05s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7272727489471436
Total elapsed time: 00h 00m 19s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [24]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

In [29]:
#This is for no.of units in a layer
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value = 8, max_value = 128, step = 8)

  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [30]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name='my_project')

In [32]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 19s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'units': 48}

In [36]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

In [39]:
#This is for selecting no.of layers
def build_model(hp):
  model = Sequential()

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')

  return model

In [42]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='selecting_layers')

In [43]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 08s


In [44]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [45]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

In [56]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1, max_value=10)):

    if counter == 0:

      model.add(
          Dense(
              hp.Int(
                  'units' + str(i), min_value=8, max_value=128,step=8),
                  activation = hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid']),
                  input_dim=8
              )
          )
      model.add(
          Dropout(
              hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
          )
      )

    else:
      model.add(
          Dense(
              hp.Int(
                  'units' + str(i), min_value=8, max_value=128,step=8),
                  activation = hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid'])
              )
          )
      model.add(
          Dropout(
              hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
          )
      )
    counter+=1

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd', 'nadam', 'adadelta']), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [57]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final_with_dropout')

In [58]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.5974025726318359

Best val_accuracy So Far: 0.5974025726318359
Total elapsed time: 00h 00m 11s


In [59]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 40,
 'activation0': 'sigmoid',
 'dropout0': 0.7,
 'optimizer': 'adam',
 'units1': 104,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 96,
 'activation2': 'tanh',
 'dropout2': 0.4,
 'units3': 120,
 'activation3': 'tanh',
 'dropout3': 0.7,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1,
 'units9': 8,
 'activation9': 'relu',
 'dropout9': 0.1}

In [60]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))